<a href="https://colab.research.google.com/github/filippovaldettaro/3Drotation/blob/master/nn_with_new_labelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)

Mounted at /content/gdrive


In [2]:
import sys
sys.path.insert(1, '/content/gdrive/Shared drives/scribbles')

import gc

In [3]:

import torch
import torchvision
import torch.nn.functional as F
from torchvision import transforms
from torchvision import datasets
from torch import nn
from torch.utils import data

import scribble_dataloader_255 as dl
import definitions as dfns

device = device = 'cpu'
if torch.cuda.is_available():
  device = 'cuda'

train_set = dl.WithScribbles('/content/gdrive/Shared drives/scribbles/mini_data',
                                                 transform = transforms.Compose([transforms.Resize([150,200]),
                                                                                 transforms.ToTensor()]),
                                                 target_transform = transforms.Compose([transforms.Resize([150,200])]),
                                                 scribble_resize = (150, 200),
                                                 orientation = 'landscape')
train_loader = torch.utils.data.DataLoader(train_set, batch_size=9)

k = 32

model = dfns.SSN_Net(k = k).to(dfns.device)

epochs = 5
super_rows, super_cols = 10, 10
dslic_iterations = 10
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_lambda = 1e-5

for epoch in range(epochs):
    total_loss = 0.
    for batch, (images, masks, scribbles) in enumerate(train_loader):
        print(str(batch) + "/" + str(len(train_loader)))
        images, masks, scribbles = images.to(dfns.device), masks.to(dfns.device), scribbles.to(dfns.device)
        loss = dfns.dslic_and_losses(model(images), masks, super_cols, super_rows, dslic_iterations, loss_lambda)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(epoch, total_loss)

0/1


/content/gdrive/Shared drives/scribbles/definitions.py:117: UserWarning: Mixed memory format inputs detected while calling the operator. The operator will output channels_last tensor even if some of the inputs are not in channels_last format. (Triggered internally at  /pytorch/aten/src/ATen/native/TensorIterator.cpp:924.)
  associations = -torch.norm(patches - superpixels, dim=2)
/content/gdrive/Shared drives/scribbles/definitions.py:124: UserWarning: Mixed memory format inputs detected while calling the operator. The operator will output contiguous tensor even if some of the inputs are in channels_last format. (Triggered internally at  /pytorch/aten/src/ATen/native/TensorIterator.cpp:918.)
  return nansum(weights * patches, inplace=True, dim=3).unsqueeze(-1), associations


0 2.465433120727539


FileNotFoundError: ignored

In [ ]:
images, mask, scribbles = next(iter(train_loader))
images, mask, scribbles = images.to(device), mask.to(device), scribbles.to(device)
Q, superpixels = dfns.dslic(model(images), super_rows, super_cols, dslic_iterations)

superpixels.permute(0,2,3,1).size()
import matplotlib.pyplot as plt

outputs = model(images)
print(outputs.size())

images.size(), mask.size(), scribbles.size(), outputs.size(), superpixels.size(), Q.size()

argQ = torch.argmax(Q,dim=-1).view(9,150,200)
argQ.size()

from sklearn.neighbors import NearestNeighbors

for example in zip(images, mask, scribbles, outputs, superpixels.squeeze(-1), argQ):
    im, m, scrib, out, sp, q = example
    

    scrib_features = out.permute(1,2,0)[~(scrib==0)].to('cpu').detach()
    scrib_labels = scrib[~(scrib==0)]

    # drop the first two sp features (correspond to position and we don't want them)
    sp = sp[:,:-2].to('cpu').detach()

    nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(scrib_features)

    _, indices = nbrs.kneighbors(sp)

    sp_labels = scrib_labels[indices]

    pixel_labels = sp_labels[q].squeeze(-1)